In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [12]:
import os
import sys
import csv
scripts_path = os.path.abspath(os.path.join('../scripts/'))
data_path = os.path.abspath(os.path.join('../data/'))
sys.path.append(scripts_path)
sys.path.append(data_path)
    

In [4]:
scripts_path

'/home/treharne/Documents/projects/SAZO/scripts'

In [157]:
T_data = data_path + '/T/160609_A_extract_d.txt'
R_data = data_path + '/4PP/160609_A.dat'

In [7]:
T_data

'/home/treharne/Documents/projects/SAZO/data/T/160609_A_extract_d.txt'

In [ ]:
f = open(path, 'rb')
reader = csv.reader(f)
data = []
for line in reader:
    #nice and pythonic here
    data.append([float(x) for x in line[0].split(' ')])

In [ ]:
len(data)

In [ ]:
trans_data = np.transpose(data)

In [150]:
def read_T_data(path):
    f = open (path, 'rb')
    reader = csv.reader(f)
    data = []
    data_dict = {}
    x, y = relabel_T_coords()
    coords = np.transpose([x,y])
   
    for i, line in zip(coords, reader):
        item = [float(x) for x in line[0].split(' ')]
        
        data_dict['%s %s' % (i[0], i[1])] = {'x': i[0], 
                                             'y': i[1],
                                             'd': item[2],
                                             'wp': item[3]}
                                            
    return data_dict


In [253]:
def read_R_data(path):
    f = open(path, 'rb')
    reader = csv.reader(f)
    headers = reader.next()
    data = []
    data_dict = {}
    x, y = relabel_T_coords()
    coords = np.transpose([x,y])
    for i, line in zip(coords, reader):
        
        
        item = []
        for x in line:
            try:
                item.append(float(x))
            except:
                item.append(None)
        
        data_dict['%s %s' % (i[0], item[1])] = {'x': i[0], 
                                                   'y': item[1],
                                                   'Rs': item[2]}
    return data_dict
                                             
        

In [254]:
def relabel_T_coords():
    x = []
    y = []
    for i in range(0, 10):
        if i % 2 == 0:
            for j in range(0, 10):
                x.append((j-4.5)*10)
                y.append((4.5-i)*10)
        else:
            for j in range(0, 10):
                x.append((4.5 - j)*10)
                y.append((4.5-i)*10)
    
    return x, y

In [258]:
extracted_T = read_T_data(T_data)
extracted_R = read_R_data(R_data)

In [257]:
def getxyz(data, param):
    count = 0
    x = []
    y = []
    z = []
    for key in data.keys():
        x.append(data.get(key, None).get('x', None))
        y.append(data.get(key, None).get('y', None))
        z.append(data.get(key, None).get(param, None))
    return x, y, z
    
    

In [237]:
def plot(data, param, res=50, scale='linear'):
    x, y, z = getxyz(data, param)
    
    xi = np.linspace(min(x), max(x), res)
    yi = np.linspace(min(y), max(y), res)
    zi = griddata(x, y, z, xi, yi)
    
    fig = figure(figsize=(9,7))
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')

    S = ax.contour(xi, yi, zi, 10, colors='k')
    CF = ax.contourf(xi, yi, zi, 50, cmap='gist_rainbow')
    ax.clabel(S, fontsize=15, fmt='%1.0f')

    ax.set_xlabel('x (mm)', fontsize=15)
    ax.set_ylabel('y (mm)', fontsize=15)
    ax.tick_params(labelsize=15)

In [270]:
def merge_dict(dict1, dict2):
    dict3 = {}
    for key in dict1.keys():
        d = dict1.get(key).copy()
        d.update(dict2.get(key))
        dict3[key] = d
    return dict3
        
        

In [271]:
merged = merge_dict(extracted_R, extracted_T)

In [272]:
merged

{'-15.0 -15.0': {'Rs': 9.242227,
  'd': 230.0,
  'wp': 1.1422117948950627,
  'x': -15.0,
  'y': -15.0},
 '-15.0 -25.0': {'Rs': 7.849931,
  'd': 200.0,
  'wp': 1.2076267039993867,
  'x': -15.0,
  'y': -25.0},
 '-15.0 -35.0': {'Rs': 17.956318,
  'd': 184.0,
  'wp': 1.177450302203987,
  'x': -15.0,
  'y': -35.0},
 '-15.0 -45.0': {'Rs': 18.255698,
  'd': 167.0,
  'wp': 1.1285693230713587,
  'x': -15.0,
  'y': -45.0},
 '-15.0 -5.0': {'Rs': 7.094726,
  'd': 244.0,
  'wp': 1.117355982718312,
  'x': -15.0,
  'y': -5.0},
 '-15.0 15.0': {'Rs': 12.141358,
  'd': 283.0,
  'wp': 1.0221322977401497,
  'x': -15.0,
  'y': 15.0},
 '-15.0 25.0': {'Rs': 8.792362,
  'd': 309.0,
  'wp': 0.9557745547017039,
  'x': -15.0,
  'y': 25.0},
 '-15.0 35.0': {'Rs': 103.687133,
  'd': 318.0,
  'wp': 0.9194786026935967,
  'x': -15.0,
  'y': 35.0},
 '-15.0 45.0': {'Rs': 12.391163,
  'd': 328.0,
  'wp': 0.9207624774854878,
  'x': -15.0,
  'y': 45.0},
 '-15.0 5.0': {'Rs': 8.146557,
  'd': 279.0,
  'wp': 1.01564182886837,

In [274]:
def get_mob(data_dict):
    for key in data_dict.keys():
        print data_dict[key].get('wp')

In [275]:
get_mob(merged)

1.19941101252
1.09001451714
1.15981007526
1.16381562537
0.996022730289
0.983772639906
1.03261658835
1.21218590765
1.12740121431
1.12856932307
1.00104734176
0.356151048596
1.19725343442
1.16672788825
1.18602306857
1.16863251653
0.855819831441
0.973073334044
1.05468038537
1.09378551411
0.983146636987
0.33020420491
1.1941753281
0.786614629486
0.859565051623
0.375943878803
0.47663973146
0.945709459975
0.341533017073
1.1771597243
0.852339556916
0.881286368577
1.11735598272
1.01564182887
1.13390244371
1.19782567177
0.815328679817
1.02213229774
1.00017759849
0.970937529248
0.938556746279
1.1931365993
0.95139690913
0.921275791389
0.995778456107
0.410053066484
1.03619661327
1.17521554723
1.03402086756
1.18761924214
1.17200426864
0.890731309578
1.02534653969
1.13863848371
1.11306651333
1.00546947487
0.859386215508
1.12927999856
1.13645363622
1.20216283471
1.03601359846
1.1422117949
0.919478602694
1.1285239272
0.952787888173
1.1774503022
0.955774554702
1.07819156378
1.1587356617
1.207626704
1.176